In [2]:
import pandas as pd
import numpy as np
import sqlalchemy as sql
from sqlalchemy import create_engine,inspect

In [3]:
# creating list of dictionaries from department_budget data

department_budget=[{"sub_dep_id":1, "sub_dep_name":"managers", "department_id":1, "budget":3000},
{"sub_dep_id":2, "sub_dep_name":"managers2", "department_id":1, "budget":1500},
{"sub_dep_id":1, "sub_dep_name":"sales support john", "department_id":2, "budget":2000},
{"sub_dep_id":2, "sub_dep_name":"sales support joe", "department_id":2, "budget":1000},
{"sub_dep_id":3, "sub_dep_name":"sales support johnson", "department_id":2, "budget":2500},
{"sub_dep_id":4, "sub_dep_name":"sales support eduards", "department_id":2, "budget":2500}]

In [4]:
# Create DataFrame department_budget from  list

department_budget=pd.DataFrame(department_budget)      
department_budget

,sub_dep_id,sub_dep_name,department_id,budget
0,1,managers,1,3000
1,2,managers2,1,1500
2,1,sales support john,2,2000
3,2,sales support joe,2,1000
4,3,sales support johnson,2,2500
5,4,sales support eduards,2,2500


In [5]:
# reading file in format json into DataFrame 

department_budget_2=pd.read_json(r'C:\Users\revital\OneDrive - Nucleix\Desktop\Data analysis Course\final task\Python\python data\raw-department-budget2.txt')


In [6]:
department_budget_2.head()

,sub_dep_id,sub_dep_name,department_id,budget
0,1,IT purchases,3,2000
1,2,IT maintenance,3,1500
2,3,IT other,3,1000


In [7]:
# reading file into DataFrame using delimiter('-')
 
department=pd.read_csv(r'C:\Users\revital\OneDrive - Nucleix\Desktop\Data analysis Course\final task\Python\python data\raw-department.txt', delimiter='-')

In [8]:
department


,department_id,department_name
0,1,General
1,2,Sales Support
2,3,IT


In [9]:
 # union of department_budget and department_budget_2 to one DataFrame

department_budget=pd.concat([department_budget,department_budget_2],axis=0).reset_index(drop=True)

In [10]:
# joining of department to department_budget 


department_budget=pd.merge(department_budget,department,how='left',on='department_id')

In [11]:
# create DataFrame of sum budget per department

department_budget=department_budget.groupby(['department_id','department_name'])['budget'].sum().reset_index()
department_budget

,department_id,department_name,budget
0,1,General,4500
1,2,Sales Support,8000
2,3,IT,4500


In [12]:
# connecting and transfer table to Postgres SQL

user='postgres'
password='postgres'
db_host = 'localhost'
db_port = '5432'
database='chinook'

connection=f"postgresql+psycopg2://{user}:{password}@{db_host}:{db_port}/{database}"
engine=create_engine(connection)

department_budget.to_sql('department_budget', engine, if_exists='replace', index=False,schema='stg')

3

In [13]:
inspector=inspect(engine)

In [14]:
invoicedate="""
               select distinct(invoicedate::date ) as date
                from stg.invoice i
                """ 
inv_date=pd.read_sql_query(invoicedate,engine)
inv_date.head()

,date
0,2019-10-15
1,2021-03-09
2,2019-09-08
3,2021-07-13
4,2020-06-14


In [15]:
# inserting min and max invoice date into variables 

start_date=inv_date['date'].min()
end_date=inv_date['date'].max()

print(start_date)
print(end_date)


2018-01-01
2022-12-22


In [16]:
import requests
from xml.etree import ElementTree

In [17]:
# API URL with date range using variables

url = (
    f"https://edge.boi.gov.il/FusionEdgeServer/sdmx/v2/data/dataflow/BOI.STATISTICS/EXR/1.0/"
    f"?c%5BDATA_TYPE%5D=OF00&c%5BBASE_CURRENCY%5D=USD&locale=he"
    f"&startPeriod={start_date}&endPeriod={end_date}"
) 

# Send a GET request to the API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the XML content
    tree = ElementTree.fromstring(response.content)
    
    # Extract relevant data from the XML tree
    data = []
    for series in tree.findall(".//Series"):
        obs = series.findall(".//Obs")
        for ob in obs:
            date = ob.attrib['TIME_PERIOD']
            value = ob.attrib['OBS_VALUE']
            data.append({"Date": date, "USD_to_NIS_Rate": float(value)})
    
    # Create a DataFrame from the parsed data
    currency = pd.DataFrame(data)
    
    # Print the DataFrame
    print(currency)
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

            Date  USD_to_NIS_Rate
0     2018-01-02            3.457
1     2018-01-03            3.460
2     2018-01-04            3.448
3     2018-01-05            3.446
4     2018-01-08            3.441
...          ...              ...
1216  2022-12-16            3.451
1217  2022-12-19            3.441
1218  2022-12-20            3.469
1219  2022-12-21            3.477
1220  2022-12-22            3.477

[1221 rows x 2 columns]


In [18]:
#checking the date column type
currency['Date'].dtypes

dtype('O')

In [19]:
#Set the Date column to datetime type 
currency['Date']=pd.to_datetime(currency['Date'])

In [20]:
# Create a DataFrame with running dates (since not all dates are in Dim_currency)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
df_dates = pd.DataFrame(date_range, columns=['running_date'])

print(df_dates)

     running_date
0      2018-01-01
1      2018-01-02
2      2018-01-03
3      2018-01-04
4      2018-01-05
...           ...
1812   2022-12-18
1813   2022-12-19
1814   2022-12-20
1815   2022-12-21
1816   2022-12-22

[1817 rows x 1 columns]


In [21]:
#merging the currency data to the running date df
temp=pd.merge(df_dates,currency,how='left',left_on=['running_date'],right_on=['Date'])

In [22]:
#counting nulls in the temp df 
temp['USD_to_NIS_Rate'].isnull().sum()

np.int64(596)

In [23]:
# filling null rate values with the previous non null value
temp['USD_to_NIS_Rate'].fillna(method='ffill', inplace=True)

C:\Users\revital\AppData\Local\Temp\ipykernel_16044\3860097610.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  temp['USD_to_NIS_Rate'].fillna(method='ffill', inplace=True)
C:\Users\revital\AppData\Local\Temp\ipykernel_16044\3860097610.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp['USD_to_NIS_Rate'].fillna(method='ffill', inplace=True)


In [24]:
# filling first null rate value with the next non null value
temp['USD_to_NIS_Rate']=temp['USD_to_NIS_Rate'].fillna(method='bfill')

C:\Users\revital\AppData\Local\Temp\ipykernel_16044\1606248295.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp['USD_to_NIS_Rate']=temp['USD_to_NIS_Rate'].fillna(method='bfill')


In [25]:
temp

,running_date,Date,USD_to_NIS_Rate
0,2018-01-01,NaT,3.457
1,2018-01-02,2018-01-02,3.457
2,2018-01-03,2018-01-03,3.460
3,2018-01-04,2018-01-04,3.448
4,2018-01-05,2018-01-05,3.446
...,...,...,...
1812,2022-12-18,NaT,3.451
1813,2022-12-19,2022-12-19,3.441
1814,2022-12-20,2022-12-20,3.469
1815,2022-12-21,2022-12-21,3.477


In [26]:
temp['USD_to_NIS_Rate']=temp['USD_to_NIS_Rate'].round(4)

In [27]:
#creating Dim_currency table with the relevant columns ftom temp df
Dim_currency=temp[['running_date','USD_to_NIS_Rate']]
Dim_currency.rename(columns={'running_date':'Date'},inplace=True)
print(Dim_currency)

           Date  USD_to_NIS_Rate
0    2018-01-01            3.457
1    2018-01-02            3.457
2    2018-01-03            3.460
3    2018-01-04            3.448
4    2018-01-05            3.446
...         ...              ...
1812 2022-12-18            3.451
1813 2022-12-19            3.441
1814 2022-12-20            3.469
1815 2022-12-21            3.477
1816 2022-12-22            3.477

[1817 rows x 2 columns]


C:\Users\revital\AppData\Local\Temp\ipykernel_16044\564298850.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dim_currency.rename(columns={'running_date':'Date'},inplace=True)


In [28]:
#testing for null values in 'USD_to_NIS_Rate' column
Dim_currency['USD_to_NIS_Rate'].isnull().sum()

np.int64(0)

In [29]:
#converting the date from datetime type to date
Dim_currency['Date']=Dim_currency['Date'].dt.date

C:\Users\revital\AppData\Local\Temp\ipykernel_16044\2907403870.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dim_currency['Date']=Dim_currency['Date'].dt.date


In [30]:
# transfer of table Dim_currency to SQL DWH

user='postgres'
password='postgres'
db_host = 'localhost'
db_port = '5432'
database='chinook'

connection=f"postgresql+psycopg2://{user}:{password}@{db_host}:{db_port}/{database}"
engine=create_engine(connection)

Dim_currency.to_sql('Dim_currency', engine, if_exists='replace', index=False,schema='chinook_dwh')



817